In [6]:
# import pandas as pd

# # Load the data from the file
# file_path = 'shippingdates/Rush Soccer 12.29 - Details.csv'  # Replace with the actual file path
# df = pd.read_csv(file_path)
# print(df.columns)
# print(df.dtypes)


# # Convert the 'Date Created' and 'Shipping Date' to datetime format
# df['Date Created'] = pd.to_datetime(df['Date Created'])
# df['Shipping Date'] = pd.to_datetime(df['Shipping Date'])

# # df.rename(columns={" Order Quantity ": "Order Quantity", " Shipped Quantity ": "Shipped Quantity", ' Unshipped Quantity ': 'Unshipped Quantity' })
# # Strip whitespace from all column names
# df.columns = df.columns.str.strip()

# # Strip whitespace from specific columns if they contain string values
# columns_to_strip = ['Order Quantity', 'Shipped Quantity', 'Unshipped Quantity']
# df[columns_to_strip] = df[columns_to_strip].apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# print(df.dtypes)
# print(df.head())
# # print(df.columns)

# # Group by 'Customer Reference' and aggregate accordingly
# grouped_df = df.groupby(['Customer Reference', 'Club Name']).agg({
#     'Date Created': 'min',        # Minimum of Date Created
#     'Order Quantity': 'sum',      # Sum of Order Quantity
#     'Shipped Quantity': 'sum',    # Sum of Shipped Quantity
#     'Unshipped Quantity': 'sum',  # Sum of Unshipped Quantity
#     'Shipping Date': 'max',       # Maximum of Shipping Date
#     'Sales Order Header Status': 'last'  # Take the first status (assuming it's consistent)
# }).reset_index()

# # Save the result to a new file
# output_file = 'aggregated_orders12.29.csv'  # Replace with the desired output file name
# grouped_df.to_csv(output_file, index=False)

# print(f"Aggregated data saved to {output_file}")


In [1]:
import pandas as pd

# -------------------------- Data Loading and Preprocessing -------------------------- #

def load_and_preprocess_data(file_path):
    """
    Loads the CSV data, preprocesses it by handling missing values, converting data types,
    and stripping whitespace from column names and string fields.

    Parameters:
    - file_path (str): Path to the CSV file.

    Returns:
    - pd.DataFrame: Preprocessed DataFrame.
    """
    try:
        # Load the data from the file
        df = pd.read_csv(file_path)
        print("Data loaded successfully.")
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None
    except pd.errors.ParserError:
        print(f"Error: Could not parse the file '{file_path}'. Please ensure it's a valid CSV.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred while loading the data: {e}")
        return None

    df['Shipping Date']=df['Shipped Date']
    df = df.drop("Shipped Date", axis = 1)
    # Display initial columns and data types
    print("Initial Columns:", df.columns.tolist())
    print("Initial Data Types:\n", df.dtypes)
    
    # Strip whitespace from all column names
    df.columns = df.columns.str.strip()

    # Replace 'N/A' strings with NaN for proper handling
    df.replace('N/A', pd.NA, inplace=True)

    # Identify categorical and numerical columns
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    categorical_cols = [col for col in categorical_cols if col != 'ID']  # Exclude 'ID' if present

    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

    # Convert 'Date Created' and 'Shipping Date' to datetime, coercing errors to NaT
    for date_col in ['Date Created', 'Shipping Date']:
        if date_col in df.columns:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        else:
            print(f"Warning: '{date_col}' column not found in the data.")

    # Convert quantity columns to numeric, coercing errors to NaN
    quantity_cols = ['Order Quantity', 'Shipped Quantity', 'Unshipped Quantity']
    for qty_col in quantity_cols:
        if qty_col in df.columns:
            df[qty_col] = pd.to_numeric(df[qty_col], errors='coerce')
        else:
            print(f"Warning: '{qty_col}' column not found in the data.")

    # Handle missing values
    # For categorical columns, we will leave NaN as is (do not fill with 'No')
    # For numerical columns, fill NaN with the median of each column
    df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

    # Ensure 'ID' is treated as a string to avoid numeric issues
    if 'ID' in df.columns:
        df['ID'] = df['ID'].astype(str)

    # Drop 'Data Type' column if present to avoid serialization issues
    if 'Data Type' in df.columns:
        df = df.drop('Data Type', axis=1)
        print("Column 'Data Type' was found and has been excluded from the analysis.")

    # Strip whitespace from specific columns if they contain string values
    for col in quantity_cols:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].str.strip()

    # Display data types after preprocessing
    print("Data Types After Preprocessing:\n", df.dtypes)
    print("First 5 Rows After Preprocessing:\n", df.head())

    return df

# -------------------------- Aggregation Logic -------------------------- #

def aggregate_orders(df):
    """
    Aggregates the orders by 'Customer Reference' and 'Club Name'. 
    Ensures that if any item within a group has a status of 'OPEN', the entire group's status is set to 'OPEN'.

    Parameters:
    - df (pd.DataFrame): The preprocessed DataFrame.

    Returns:
    - pd.DataFrame: Aggregated DataFrame.
    """
    def aggregate_status(status_series):
        """
        Aggregates the 'Sales Order Header Status'.
        If any status in the group is 'OPEN', return 'OPEN'.
        Otherwise, return the most common status or another logic as needed.
        """
        if 'OPEN' in status_series.values:
            return 'OPEN'
        else:
            # If there are multiple statuses and 'OPEN' is not present, define your own logic
            # For example, return the mode (most frequent status)
            return status_series.mode().iloc[0] if not status_series.mode().empty else 'SHIPPED'

    # Perform the aggregation
    grouped_df = df.groupby(['Customer Reference', 'Club Name']).agg({
        'Date Created': 'min',                        # Earliest Date Created
        'Order Quantity': 'sum',                      # Total Order Quantity
        'Shipped Quantity': 'sum',                    # Total Shipped Quantity
        'Unshipped Quantity': 'sum',                  # Total Unshipped Quantity
        'Shipping Date': 'max',                       # Latest Shipping Date
        'Sales Order Header Status': aggregate_status  # Custom aggregation for status
#         'Home Long Sleeve': 'max',
#         'Away Long Sleeve': 'max'
        
    }).reset_index()

    print("Aggregated Data Types:\n", grouped_df.dtypes)
    print("First 5 Rows of Aggregated Data:\n", grouped_df.head())

    return grouped_df

# -------------------------- Main Execution -------------------------- #

def main():
    # Define file paths
    input_file = 'shippingdates/Rush Soccer 5.4 - Details.csv'  # Replace with the actual file path
    output_file = 'shippingdates/aggregated_orders5.4.csv'                   # Desired output file name

    # Load and preprocess data
    df = load_and_preprocess_data(input_file)
    if df is None:
        return  # Exit if data loading failed

    # Aggregate orders
    aggregated_df = aggregate_orders(df)

    # Save the aggregated data to a new CSV file
    try:
        aggregated_df.to_csv(output_file, index=False)
        print(f"Aggregated data saved to '{output_file}'.")
    except Exception as e:
        print(f"An error occurred while saving the aggregated data: {e}")

if __name__ == "__main__":
    main()


Data loaded successfully.
Initial Columns: ['Customer Reference', 'Club Name', 'Date Created', 'Sold TO Name', 'Sold TO Email', 'Ship TO Name', 'Order Quantity', 'Shipped Quantity', 'Unshipped Quantity', 'Tracking Number', 'Sales Order Header Status', 'Material Code', 'Description', 'Size', 'Shipping Date']
Initial Data Types:
 Customer Reference            int64
Club Name                    object
Date Created                 object
Sold TO Name                 object
Sold TO Email                object
Ship TO Name                 object
Order Quantity                int64
Shipped Quantity              int64
Unshipped Quantity            int64
Tracking Number              object
Sales Order Header Status    object
Material Code                object
Description                  object
Size                         object
Shipping Date                object
dtype: object
Data Types After Preprocessing:
 Customer Reference                    int64
Club Name                            ob

In [5]:
import pandas as pd

# Read the CSV file into a DataFrame.
df = pd.read_csv("shippingdates/Rush Soccer 4.27 - Rush Details Category 2025.csv")

# OPTIONAL: To restrict analysis to a specific club, e.g., "Rush Montana", uncomment:
df = df[df["Club Name"] == "Rush Nevada"]
df = df[df["Category"].isin(["Field Players Mandatory Kit", "Goalkeepers Mandatory Kit","Competitive Items"])]

# Convert "Order Date" to datetime and filter for orders in 2025.
df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")
df = df[df["Order Date"].dt.year == 2025]

# Define the long sleeve product descriptions.
home_ls = "NEVADA RUSH BROOKLYN II RUSH SOCCER PYRAMIDS LONG SLEEVE MATCH JERSEY PROMO BLUE BLACK WHITE"
away_ls = "NEVADA RUSH BROOKLYN II RUSH SOCCER SCATTERED SHARDE LONG SLEEVE MATCH JERSEY WHITE PROMO GREY BLACK"

# Function to compute long sleeve status and counts from a list of product names.
def compute_ls_status_player(product_names):
    # Convert the list into a pandas Series for easier aggregation.
    s = pd.Series(product_names)
    count_home = s.isin([home_ls]).sum()
    count_away = s.isin([away_ls]).sum()
    total = count_home + count_away
    if total == 0:
        status = "Did Not Purchase Long Sleeve"
    elif count_home > 0 and count_away > 0:
        status = f"Purchased Both Long Sleeves ({total})"
    elif count_home > 0:
        status = f"Purchased Only Home Long Sleeve ({count_home})"
    elif count_away > 0:
        status = f"Purchased Only Away Long Sleeve ({count_away})"
    else:
        status = "Did Not Purchase Long Sleeve"
    return status, count_home, count_away

# Group by "Player ID" so that if a player purchases in multiple orders, they are aggregated together.
grouped = df.groupby("Player ID").agg({
    "Order ID": lambda x: " ; ".join(x.astype(str).unique()),
    "Order Date": "min",             # earliest order date for that player
    "Customer Email": "first",       # assuming one email per player
    "Club Name": "first",
    "Player Name": "first",
    "Product Name": lambda x: list(x)  # aggregate all product names into a list
}).reset_index()

# Compute long sleeve status and counts for each player.
grouped[["Long Sleeve Status", "Home Count", "Away Count"]] = grouped["Product Name"].apply(
    lambda prod_list: pd.Series(compute_ls_status_player(prod_list))
)

# Print out unique players (Player ID, Customer Email, Player Name, and status) for players who purchased any long sleeve.
purchased_ls = grouped[grouped["Long Sleeve Status"] != "Did Not Purchase Long Sleeve"]
print("Players who purchased long sleeve jerseys:")
print(purchased_ls[["Player ID", "Customer Email", "Player Name", "Long Sleeve Status", "Home Count", "Away Count"]])

# Print summary statistics: count of unique players by Long Sleeve Status.
summary = grouped.groupby("Long Sleeve Status").size()
print("\nSummary Statistics (Unique Players by Long Sleeve Status):")
print(summary)

# Print total counts of Home and Away long sleeve jerseys purchased across all players.
total_home = grouped["Home Count"].sum()
total_away = grouped["Away Count"].sum()
print(f"\nTotal Home Long Sleeve Jerseys Purchased: {total_home}")
print(f"Total Away Long Sleeve Jerseys Purchased: {total_away}")

# Write the aggregated order summary to CSV in the specified directory.
grouped.to_csv("LongSleeveOrders/4.13Order_Summary_with_ls_status.csv", index=False)

# Optionally, merge the summary information back into the original DataFrame by Player ID.
df = df.merge(grouped[["Player ID", "Long Sleeve Status", "Home Count", "Away Count"]],
              on="Player ID", how="left")
df.to_csv("LongSleeveOrders/Rush Soccer 4.13 - Details_with_ls_status.csv", index=False)

print("\nFiles saved: '4.13Order_Summary_with_ls_status.csv' and 'Rush Soccer 4.13 - Details_with_ls_status.csv'")


# import pandas as pd

# # Read the CSV file into a DataFrame. Adjust the filename as needed.
# df = pd.read_csv("shippingdates/Rush Soccer 3.2 NEW - Rush Details Category 2025.csv")

# # OPTIONAL: To restrict analysis to a specific club (e.g., "Rush Montana"), uncomment:
# df = df[df["Club Name"] == "Rush Montana"]

# # Convert "Order Date" to datetime and filter for orders placed in 2025.
# df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")
# df = df[df["Order Date"].dt.year == 2025]

# # Define the long sleeve product descriptions.
# home_ls = "MONTANA RUSH BROOKLYN II RUSH SOCCER PYRAMIDS LONG SLEEVE MATCH JERSEY PROMO BLUE BLACK WHITE"
# away_ls = "MONTANA RUSH BROOKLYN II RUSH SOCCER SCATTERED SHARDE LONG SLEEVE MATCH JERSEY WHITE PROMO GREY BLACK"

# # Function to compute long sleeve status and counts from a list of product names.
# def compute_ls_status(descriptions):
#     # Convert the list to a pandas Series for easier calculation.
#     desc_series = pd.Series(descriptions)
#     count_home = desc_series.isin([home_ls]).sum()
#     count_away = desc_series.isin([away_ls]).sum()
#     total = count_home + count_away
#     if total == 0:
#         status = "Did Not Purchase Long Sleeve"
#     elif count_home > 0 and count_away > 0:
#         status = f"Purchased Both Long Sleeves ({total})"
#     elif count_home > 0:
#         status = f"Purchased Only Home Long Sleeve ({count_home})"
#     elif count_away > 0:
#         status = f"Purchased Only Away Long Sleeve ({count_away})"
#     else:
#         status = "Did Not Purchase Long Sleeve"
#     return status, count_home, count_away

# # Group by "Order ID" and "Player ID" so that we aggregate items per player in an order.
# order_summary = df.groupby(["Customer Reference", "Player ID"]).agg({
#     "Order Date": "first",           # Take the first order date per group.
#     "Customer Email": "first",       # Assume each order/player has one email.
#     "Club Name": "first",
#     "Player Name": "first",
#     "Product Name": lambda x: list(x)  # Aggregate all product names for this order/player.
# }).reset_index()

# # Compute long sleeve status and counts for each group.
# order_summary[["Long Sleeve Status", "Home Count", "Away Count"]] = order_summary["Product Name"].apply(
#     lambda desc: pd.Series(compute_ls_status(desc))
# )

# # Print unique orders (Order ID, Player ID, Email, and status) that purchased any long sleeve.
# purchased_ls = order_summary[order_summary["Long Sleeve Status"] != "Did Not Purchase Long Sleeve"]
# print("Orders that purchased any long sleeve:")
# print(purchased_ls[["Customer Reference", "Player ID", "Customer Email", "Player Name", "Long Sleeve Status", "Home Count", "Away Count"]])

# # Print summary statistics: count of unique orders by Long Sleeve Status.
# summary = order_summary.groupby("Long Sleeve Status").size()
# print("\nSummary Statistics (Unique Orders by Long Sleeve Status):")
# print(summary)

# # Print total counts of Home and Away long sleeve jerseys across all orders.
# total_home = order_summary["Home Count"].sum()
# total_away = order_summary["Away Count"].sum()
# print(f"\nTotal Home Long Sleeve Jerseys Purchased: {total_home}")
# print(f"Total Away Long Sleeve Jerseys Purchased: {total_away}")

# # Write the order summary DataFrame to CSV in the designated directory.
# order_summary.to_csv("LongSleeveOrders/3.2Order_Summary_with_ls_status.csv", index=False)

# # Optionally, merge the computed status back into the original DataFrame for a full export.
# df = df.merge(order_summary[["Customer Reference", "Player ID", "Long Sleeve Status", "Home Count", "Away Count"]],
#               on=["Customer Reference", "Player ID"], how="left")
# df.to_csv("LongSleeveOrders/Rush Soccer 3.2 - Details_with_ls_status.csv", index=False)

# print("\nFiles saved: '3.2Order_Summary_with_ls_status.csv' and 'Rush Soccer 3.2 - Details_with_ls_status.csv'")


Players who purchased long sleeve jerseys:
    Player ID              Customer Email Player Name  \
4    623334.0    mvallesteros05@gmail.com        None   
11   623343.0             double2@cox.net        None   
24   623396.0     Rachel_Donn@hotmail.com        None   
25   623397.0    gabrielanano16@yahoo.com        None   
32   623410.0  LRisiglione@NevadaRush.com        None   
34   623425.0             ibarrjg@msn.com        None   
35   623426.0   marybarnett1027@gmail.com        None   
36   623427.0          juve5586@gmail.com        None   
37   623431.0     jserrano@nevadarush.com        None   
43   623486.0     jserrano@nevadarush.com        None   
44   623492.0  LRisiglione@NevadaRush.com        None   
46   623499.0        roberekson@gmail.com        None   
50   623510.0         sgetaneh9@gmail.com        None   
52   623522.0         sgetaneh9@gmail.com        None   

                     Long Sleeve Status  Home Count  Away Count  
4   Purchased Only Away Long Sleeve

In [2]:
617107
# import pandas as pd

# # ------------------------- 1. Load the Data ------------------------- #

# # Load the data from the file
# file_path = 'shippingdates/Rush Soccer 12.15 - Details.csv'  # Replace with the actual file path
# df = pd.read_csv(file_path)

# # Display column names and data types
# print("Column Names:", df.columns.tolist())
# print("\nData Types Before Cleaning:\n", df.dtypes)

# # ------------------------- 2. Clean Column Names ------------------------- #

# # Strip whitespace from all column names
# df.columns = df.columns.str.strip()

# # If needed, rename columns to remove internal spaces or correct names
# # Example:
# # df.rename(columns={" Order Quantity ": "Order Quantity", 
# #                    " Shipped Quantity ": "Shipped Quantity", 
# #                    ' Unshipped Quantity ': 'Unshipped Quantity'}, inplace=True)

# # ------------------------- 3. Convert Quantity Columns to Numeric ------------------------- #

# # Define the quantity columns
# quantity_columns = ['Order Quantity', 'Shipped Quantity', 'Unshipped Quantity']

# # Check if these columns exist
# missing_columns = [col for col in quantity_columns if col not in df.columns]
# if missing_columns:
#     print(f"Warning: The following quantity columns are missing: {missing_columns}")
#     # Optionally, handle missing columns here (e.g., create them with default values)
# else:
#     # Convert to numeric, coercing errors to NaN
#     for col in quantity_columns:
#         df[col] = pd.to_numeric(df[col], errors='coerce')

#     # Replace NaN values with 0
#     df[quantity_columns] = df[quantity_columns].fillna(0)

#     # Verify data types after conversion
#     print("\nData Types After Conversion:\n", df.dtypes)

#     # Display the first few rows after cleaning
#     print("\nData Sample After Cleaning:")
#     print(df.head())

#     # ------------------------- 4. Perform Aggregation ------------------------- #

#     # Group by 'Customer Reference' and 'Club Name' and aggregate
#     grouped_df = df.groupby(['Customer Reference', 'Club Name']).agg({
#         'Date Created': 'min',                        # Minimum of Date Created
#         'Order Quantity': 'sum',                      # Sum of Order Quantity
#         'Shipped Quantity': 'sum',                    # Sum of Shipped Quantity
#         'Unshipped Quantity': 'sum',                  # Sum of Unshipped Quantity
#         'Shipping Date': 'max',                       # Maximum of Shipping Date
#         'Sales Order Header Status': 'last'           # Take the last status
#     }).reset_index()

#     # Display the aggregated data
#     print("\nAggregated Data Sample:")
#     print(grouped_df.head())

#     # ------------------------- 5. Save Aggregated Data ------------------------- #

#     # Save the aggregated data to a new CSV file
#     output_file = 'aggregated_orders12.15.csv'  # Replace with desired output file name
#     grouped_df.to_csv(output_file, index=False)

#     print(f"\nAggregated data saved to {output_file}")


In [21]:
# # packages_shipped_analysis.ipynb

# import pandas as pd

# # -------------------------- Step 1: Load the Data -------------------------- #

# # Define the file path
# file_path = 'shippingdates/Rush Soccer 11.24.24 - Details.csv'  # Replace with your actual file path

# # Load the CSV file into a pandas DataFrame
# # Assuming the CSV is tab-separated based on the sample data
# try:
#     df = pd.read_csv(file_path, sep='\t', engine='python')
#     print("Data loaded successfully.")
# except Exception as e:
#     print(f"Error loading data: {e}")

# # -------------------------- Step 2: Inspect the Data -------------------------- #

# # Display the first few rows to understand the data structure
# print("First 5 rows of the dataset:")
# print(df.head())

# # Display the column names and their data types
# print("\nColumn Names:")
# print(df.columns)
# print("\nData Types:")
# print(df.dtypes)

# # -------------------------- Step 3: Clean Column Names -------------------------- #

# # Strip leading and trailing whitespace from all column names
# df.columns = df.columns.str.strip()

# # Verify column names after stripping
# print("\nCleaned Column Names:")
# print(df.columns)

# # -------------------------- Step 4: Clean Specific Columns -------------------------- #

# # Define columns that may contain whitespace and need to be stripped
# columns_to_strip = ['Order Quantity', 'Shipped Quantity', 'Unshipped Quantity']

# # Strip whitespace from these columns if they are of object type (strings)
# df[columns_to_strip] = df[columns_to_strip].apply(
#     lambda x: x.str.strip() if x.dtype == "object" else x
# )

# # Convert 'Shipped Quantity' and 'Unshipped Quantity' to numeric types
# # Replace non-numeric entries with 0
# df['Shipped Quantity'] = pd.to_numeric(df['Shipped Quantity'], errors='coerce').fillna(0).astype(int)
# df['Unshipped Quantity'] = pd.to_numeric(df['Unshipped Quantity'], errors='coerce').fillna(0).astype(int)

# # Verify the changes
# print("\nData Types After Conversion:")
# print(df.dtypes)

# # -------------------------- Step 5: Convert Date Columns -------------------------- #

# # Convert 'Date Created' and 'Shipping Date' to datetime format
# # Coerce errors to NaT (Not a Time) for invalid dates
# df['Date Created'] = pd.to_datetime(df['Date Created'], errors='coerce', format='%m/%d/%Y')
# df['Shipping Date'] = pd.to_datetime(df['Shipping Date'], errors='coerce', format='%m/%d/%Y')

# # Verify the conversion
# print("\nDate Columns After Conversion:")
# print(df[['Date Created', 'Shipping Date']].head())

# # -------------------------- Step 6: Handle Invalid 'Shipping Date' Entries -------------------------- #

# # Identify rows with invalid 'Shipping Date' (NaT)
# invalid_shipping_dates = df['Shipping Date'].isna()
# print(f"\nNumber of rows with invalid 'Shipping Date': {invalid_shipping_dates.sum()}")

# # Option 1: Remove rows with invalid 'Shipping Date'
# df_clean = df.dropna(subset=['Shipping Date']).copy()
# print(f"Number of rows after removing invalid 'Shipping Date': {df_clean.shape[0]}")

# # Optionally, you can choose to fill invalid 'Shipping Date' with 'Date Created' or another default date
# # Uncomment the following lines if you prefer this approach
# # df['Shipping Date'] = df['Shipping Date'].fillna(df['Date Created'])
# # df_clean = df.dropna(subset=['Shipping Date']).copy()

# # -------------------------- Step 7: Extract Month-Year from 'Shipping Date' -------------------------- #

# # Create a new column 'Month-Year' in 'MMMM YYYY' format (e.g., July 2024)
# df_clean['Month-Year'] = df_clean['Shipping Date'].dt.strftime('%B %Y')

# # Verify the new column
# print("\nSample 'Month-Year' Entries:")
# print(df_clean[['Shipping Date', 'Month-Year']].head())

# # -------------------------- Step 8: Remove Duplicate Tracking Numbers -------------------------- #

# # Assuming each 'Tracking Number' uniquely identifies a package, remove duplicates
# # If 'Tracking Number' is not unique per package, adjust accordingly
# df_unique = df_clean.drop_duplicates(subset=['Tracking Number'])

# # Verify the removal of duplicates
# print(f"\nNumber of unique packages after removing duplicates: {df_unique.shape[0]}")

# # -------------------------- Step 9: Group by 'Month-Year' and Count Unique Packages -------------------------- #

# # Group by 'Month-Year' and count unique 'Tracking Number' to get the number of packages shipped each month
# packages_shipped = df_unique.groupby('Month-Year')['Tracking Number'].nunique().reset_index()

# # Rename the columns for clarity
# packages_shipped.columns = ['Month-Year', 'Unique Packages Shipped']

# # -------------------------- Step 10: Sort the Results Chronologically -------------------------- #

# # Convert 'Month-Year' back to datetime for sorting
# packages_shipped['Month-Year-Date'] = pd.to_datetime(packages_shipped['Month-Year'], format='%B %Y')

# # Sort by the new datetime column
# packages_shipped = packages_shipped.sort_values('Month-Year-Date')

# # Drop the auxiliary datetime column
# packages_shipped = packages_shipped.drop('Month-Year-Date', axis=1)

# # -------------------------- Step 11: Display the Results -------------------------- #

# print("\nNumber of Packages Shipped Each Month:")
# print(packages_shipped)

# # -------------------------- Step 12: (Optional) Save the Results to CSV -------------------------- #

# # Define the output file path
# output_file = 'packages_shipped_per_month.csv'

# # Save the DataFrame to a new CSV file
# packages_shipped.to_csv(output_file, index=False)
# print(f"\nAggregated data saved to {output_file}")


In [14]:
import pandas as pd
import numpy as np
import shutil

# ================================
# 1. Backup the Original CSV File (Optional)
# ================================

original_file = 'shippingdates/Capelli Historical 2023 Data - Details.csv'  # Original data file
aggregated_file = 'Capelli_aggregated_orders.csv'  # Aggregated data file

backup_file = 'shippingdates/Capelli_Historical_2023_Data_Backup.csv'
shutil.copyfile(original_file, backup_file)
print(f"Backup created at {backup_file}")

# ================================
# 2. Load and Inspect the Data
# ================================

# Load the data
df = pd.read_csv(original_file)

# Display initial columns and data types
print("Columns in the DataFrame:")
print(df.columns.tolist())
print("\nData Types:")
print(df.dtypes)
print("\nFirst 5 Rows:")
print(df.head())

# ================================
# 3. Clean and Standardize Column Names
# ================================

# Function to clean column names
def clean_column_names(columns):
    columns = columns.str.lower().str.strip()  # Convert to lowercase and strip whitespace
    columns = columns.str.replace('[?/(),]', '', regex=True)  # Remove special characters
    columns = columns.str.replace(' ', '_')  # Replace spaces with underscores
    return columns

# Apply the function to clean column names
df.columns = clean_column_names(df.columns)
print("\nStandardized Columns:")
print(df.columns.tolist())

# ================================
# 4. Select Only Required Columns
# ================================

# Define the required columns
required_columns = ['order_id', 'order_date', 'ship_date', 'product_quantity', 'club_name']

# Check if all required columns exist
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"Error: The following required columns are missing from the DataFrame: {missing_cols}")
else:
    # Select only the required columns
    df = df[required_columns]
    print("\nDataFrame after selecting required columns:")
    print(df.head())

    # ================================
    # 5. Clean Specific Columns
    # ================================

    # Strip whitespace from 'order_id' if it's a string
    if df['order_id'].dtype == object:
        df['order_id'] = df['order_id'].str.strip()

    # Convert 'order_date' and 'ship_date' to datetime format
    df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
    df['ship_date'] = pd.to_datetime(df['ship_date'], errors='coerce')

    # Convert 'product_quantity' to numeric and handle missing values
    df['product_quantity'] = pd.to_numeric(df['product_quantity'], errors='coerce').fillna(0).astype(int)

    # Display first 5 rows after cleaning
    print("\nFirst 5 Rows after Cleaning:")
    print(df.head())

    # Check for missing values in key columns
    print("\nMissing Values in Key Columns:")
    print(df.isnull().sum())

    # Handle missing values if necessary
    # For example, drop rows where 'order_id' is missing
    df = df.dropna(subset=['order_id'])

    # ================================
    # 6. Aggregate the Data
    # ================================

    # Group by 'order_id' and aggregate
    grouped_df = df.groupby('order_id').agg({
        'club_name': 'first',              # Assuming one club per order
        'order_date': 'min',               # Earliest order date
        'ship_date': 'max',                # Latest ship date
        'product_quantity': 'sum'          # Total products ordered
    }).reset_index()

    # Rename columns for clarity
    grouped_df.rename(columns={
        'club_name': 'Club Name',
        'order_date': 'First Order Date',
        'ship_date': 'Latest Ship Date',
        'product_quantity': 'Total Products Ordered'
    }, inplace=True)

    # Display the aggregated data
    print("\nAggregated Data:")
    print(grouped_df.head())

    # ================================
    # 7. Save the Aggregated Data Back to the Same CSV File
    # ================================

    # Save the aggregated data to the same CSV file
    grouped_df.to_csv(aggregated_file, index=False)
    print(f"\nAggregated data saved to {aggregated_file}")


Backup created at shippingdates/Capelli_Historical_2023_Data_Backup.csv
Columns in the DataFrame:
['Order ID', 'Order Date', 'Ship Date', 'Customer Email', 'Customer Name', 'Club Name', 'Product Sku', 'Material', 'Product Name', 'Product Quantity']

Data Types:
Order ID             int64
Order Date          object
Ship Date           object
Customer Email      object
Customer Name       object
Club Name           object
Product Sku         object
Material            object
Product Name        object
Product Quantity     int64
dtype: object

First 5 Rows:
   Order ID  Order Date Ship Date             Customer Email  \
0   1129547  12/13/2023  1/2/2024  mychelle_gaynor@yahoo.com   
1   1129547  12/13/2023  1/2/2024  mychelle_gaynor@yahoo.com   
2   1129547  12/13/2023  1/2/2024  mychelle_gaynor@yahoo.com   
3   1131083  12/26/2023  1/2/2024      heidir392@hotmail.com   
4   1131083  12/26/2023  1/2/2024      heidir392@hotmail.com   

       Customer Name       Club Name                  

In [7]:
# import pandas as pd
# import numpy as np
# import shutil

# # ================================
# # 1. Backup the Original CSV File
# # ================================

# input_file = 'Capelli_aggregated_orders.csv'
# backup_file = 'Capelli_aggregated_orders_backup.csv'

# # Create a backup
# shutil.copyfile(input_file, backup_file)
# print(f"Backup created at {backup_file}")

# # ================================
# # 2. Load and Inspect the Data
# # ================================

# # Load the data
# df = pd.read_csv(input_file)

# # Display the first few rows and column information
# print("Initial Columns:")
# print(df.columns.tolist())
# print("\nData Types:")
# print(df.dtypes)
# print("\nFirst 5 Rows:")
# print(df.head())
